# Probability Distributions.

Sympy has a stats module, but it has some pretty brutal limitations. For example, if I want to compute the expected value of a binomial random variable, I can't unless the number of trials is an integer.

In [1]:
# example of sympy.stats limitations
import sympy as sp
from sympy.stats import Binomial as Bin
from sympy.stats import E as Ex

n = sp.Symbol('n', positive=True, integer=True)
p = sp.Symbol('p', positive=True)
X = Bin('X', n, p) 

try:
    Ex(X) # fails with n is not an integer
except:
    print('E(X) fails')

E(X) fails


I needed to do this, and more, for another project, so I created a distributions module `distributions.py` which allows you to work out probabilities, moments, expected values, and Fisher information for a number of common statistical distributions.

It works by defining the probability desnity function and the moment generating function for all the distributions. Sometime in the future, I'll attempt to work these out as sympy calculations rather than define them. However, this could prove difficult. For example, sympy can integrate `exp(-x**2)` but not `exp(-(x-mu)**2)`.

To get started, `import` the `distributions.py` file. You'll need to have `sympy` installed.

In [2]:
# import some common functions. Note that log is like sympy.log but forces expansion
from distributions import E, pr, log

# code to display a tuple of sympy expr as math
import IPython

def showmath(*args):
    return IPython.display.Math(',\\;'.join([sp.latex(a) for a in args]))

# variance can be defined based on E
def Var(x):
    return sp.factor(E(x**2)-E(x)**2)

## Binomial distribution.

To create a binomial distribution, call `Binomial'. This takes three parameters:

* the name of the symbol. Typically you will want to use 'y', as this is common when referring to a variable taken from a Gaussian/normal distribution.
* the probability of success. This defaults to a symbol `p`.
* the number of attempts. This defaults to a symbol `n`

The probability and count are stored as properties `p` and `n` of the Binomial symbol.

[wikipedia](https://en.wikipedia.org/wiki/Binomial_distribution)

In [3]:
from distributions import Binomial
y = Binomial('y')
pr(y)

p**y*(1 - p)**(-y + n)*binomial(y, n)

In [4]:
showmath(E(y),E(y**2),Var(y))

<IPython.core.display.Math object>

In [5]:
# the Fisher information with respect to the probability p
E(sp.diff(log(pr(y)),y.p)**2)

-n/(p*(p - 1))

## Gaussian distribution.

To create a gaussian distribution object, call `Gaussian`. This takes three parameters:

* the name of the symbol. Typically you will want to use 'x' or 'z', as this is common when referring to a variable taken from a Gaussian/normal distribution.
* the mean. This defaults to a symbol `mu`.
* the standard deviation. This defaults to a symbol `sigma`

The mean and variance are stored as properties `mu` and `sigma` of the Gaussian symbol.

[wikipedia](https://en.wikipedia.org/wiki/Normal_distribution) 

In [6]:
# create a gaussian r.v. with mean mu, standard deviation sigma:
from distributions import Gaussian
z = Gaussian('z')
pr(z)

sqrt(2)*exp(-(z - mu)**2/(2*sigma**2))/(2*sqrt(pi)*sqrt(sigma**2))

In [7]:
showmath(E(z), E(z**2), Var(z))

<IPython.core.display.Math object>

In [8]:
# the Fisher information with respect to the mean
E(sp.diff(log(pr(z)),'mu')**2)

sigma**(-2)

## Poisson distribution.

To create a Poisson distribution, call `Poisson'. This takes two parameters:

* the name of the symbol. 
* the rate. This defaults to a symbol `lambda`.

The rate symbol is stored as a property `_lambda` of the Poisson distribution symbol.

[wikipedia](https://en.wikipedia.org/wiki/Poisson_distribution)

In [9]:
from distributions import Poisson
y = Poisson('y')
pr(y)

lambda**y*exp(-lambda)/factorial(y)

In [10]:
showmath(E(y),E(y**2),Var(y))

<IPython.core.display.Math object>

In [11]:
# the Fisher information with respect to the rate
E(sp.diff(log(pr(y)),y._lambda)**2)

1/lambda

## Negative Binomial distribution.

To create a negative binomial distribution, call `NegativeBinomial`. This takes three parameters:

* the name of the symbol. 
* the probability. This defaults to a symbol `p`.
* the count. This defaults to the symbol 'k'

The rate and count symbols are stored as properties `p` and `k` of the distribution symbol.

[wikipedia](https://en.wikipedia.org/wiki/Negative_binomial_distribution)

In [12]:
from distributions import NegativeBinomial
x = NegativeBinomial('x')
pr(x)

p**x*(1 - p)**r*binomial(x + r - 1, x)

In [13]:
showmath(E(x), E(x**2), Var(x))

<IPython.core.display.Math object>

In [14]:
# the Fisher information with respect to the probability
E(sp.diff(log(pr(x)),x.p)**2)

r/(p*(p - 1)**2)

## Geometric distribution.

To create a geometric distribution, call `Geometric`. This takes two parameters:

* the name of the symbol
* the probability: this defaults to the symbol `p`

The probability is stored as property `p` of the distribution.

[wikipedia](https://en.wikipedia.org/wiki/Geometric_distribution)


In [15]:
from distributions import Geometric
y = Geometric('y')
pr(y)

p*(1 - p)*(y - 1)

In [16]:
showmath(E(y),E(y**2),Var(y))

<IPython.core.display.Math object>

In [17]:
# Fisher info
E(sp.diff(log(pr(y)),y.p)**2)

(2*p - 1)**2/(p**2*(p - 1)**2)

## Gamma Distribution.

To create a Gamma distribution use `Gamma`. It takes three parameters:

* the name of the symbol
* the shape parameter k: this defaults to a symbol `k`
* the scale parameter theta: this defaults to the symbol `theta`

The shape and scale parameters are stored as properties `k` and `theta` of the gamma object. When $k=1$ this is an exponential distribution.

Alternatively, `Gamma_alt(name, alpha, beta)` lets you specify the distribution with shape $\alpha=k$ and rate parameter $\beta=1/\theta$

[wikipedia](https://en.wikipedia.org/wiki/Gamma_distribution)

In [18]:
from distributions import Gamma
x = Gamma('x')
pr(x)

x**(k - 1)*theta**(-x)*exp(-x/theta)/gamma(x)

In [19]:
showmath(E(x),E(x**2),Var(x))

<IPython.core.display.Math object>

In [20]:
# Fisher info with respect to theta
E(sp.diff(log(pr(x)),x.theta)**2)

k*(k + 1)*(theta - 1)**2/theta**2

In [21]:
from distributions import Gamma_alt
x = Gamma_alt('x')
pr(x)

x**(alpha - 1)*(1/beta)**(-x)*exp(-x*beta)/gamma(x)

In [22]:
showmath(E(x),E(x**2),Var(x))

<IPython.core.display.Math object>

In [23]:
# Fisher info wrt beta
E(sp.diff(log(pr(x)),sp.Symbol('beta'))**2)

alpha*(alpha + 1)*(beta - 1)**2/beta**4

## The Inverse Gaussian distribution.

Use `InverseGaussian`. It takes three parameters

* name
* the mean `mu`
* the scale `_lambda`

[wikipedia](https://en.wikipedia.org/wiki/Inverse_Gaussian_distribution)

In [24]:
from distributions import InverseGaussian
x = InverseGaussian('x')
pr(x)

sqrt(2)*sqrt(lambda)*exp(-lambda*(x - mu)**2/(2*x*mu**2))/(2*x**(3/2)*sqrt(pi))

In [25]:
showmath(E(x),E(x**2),Var(x))

<IPython.core.display.Math object>

In [26]:
# Fisher info with respect to mu
E(sp.diff(log(pr(x)),x.mu)**2)

lambda/mu**3

## Others.
The module also has uniform and chi-squared distributions, but chi-squared is not fully supported.

In [27]:
from distributions import Uniform # over range [a...b]
u = Uniform('u')
pr(u)

1/(-a + b)

In [28]:
# NB: this is slow because it needs to call sympy.limit
showmath(E(u),E(u**2),Var(u))

<IPython.core.display.Math object>

In [29]:
# Fisher info with respect to a, b
showmath(E(sp.diff(log(pr(u)),'a')**2), E(sp.diff(log(pr(u)),'b')**2))

<IPython.core.display.Math object>